<a href="https://colab.research.google.com/github/sachinkun21/Stock_price_prediction/blob/master/IOCL_Predictions_in_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Covering Long Short-Term Memory (LSTM) networks in Python and how we can use them to make predictions!**

In this tutorial, you will see how we can use a time-series model known as Long Short-Term Memory. LSTM models are powerful, especially for retaining a long-term memory, by design, as you will see later. You'll tackle the following topics in this tutorial:
- Understand why would you need to be able to predict stock price movements;
- Download the data - You will be using stock market data gathered from Yahoo finance;
- Split train-test data and also perform some data normalization;
- Go over and apply a few averaging techniques that can be used for one-step ahead predictions;
- Motivate and briefly discuss an LSTM model as it allows to predict more than one-step ahead;
- Predict and visualize future stock market with current data.

The only pre-requisite to this tutorial is that you have a basic understanding of Deep Learning implemnetation using python.

Let's Start!!!

### **Why Do You Need Time Series Models?**
You would like to model stock prices correctly, so as a stock buyer you can reasonably decide when to buy stocks and when to sell them to make a profit. 

This is where time series modelling comes in. You need a robust model that can look at the history of a sequence of data and correctly predict what the future elements of the sequence are going to be.

Using machine learning techniques, we will try to achieve this, develop and test the model in the upcoming sections.

Warning: Stock market prices are highly unpredictable and volatile. This means that there are no consistent patterns in the data that allow you to model stock prices over time near-perfectly. Don't take it from me, take it from Princeton University economist Burton Malkiel, who argues in his 1973 book, "A Random Walk Down Wall Street," that if the market is truly efficient and a share price reflects all factors immediately as soon as they're made public, a blindfolded monkey throwing darts at a newspaper stock listing should do as well as any investment professional.

However, let's not go all the way believing that this is just a stochastic or random process and that there is no hope for machine learning. Let's see if you can at least model the data, so that the predictions you make correlate with the actual behavior of the data. In other words, you don't need the exact stock values of the future, but the stock price movements (that is, if it is going to rise of fall in the near future)

**Importing libraries**

In [1]:
# Make sure that you have all these libaries available to run the code successfully

from pandas_datareader import data
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import urllib.request, json
import os
import numpy as np
import tensorflow as tf # This code has been tested with TensorFlow 1.6
from sklearn.preprocessing import MinMaxScaler

### **Downloading the Data**
You will be using data from the following sources:

1. **Alpha Vantage:** Before you start, however, you will first need an API key, which you can obtain for free [here](https://www.alphavantage.co/support/#api-key). After that, you can assign that key to the api_key variable.

2. **Kaggle:** Download the kaggle data from this [page](https://www.kaggle.com/borismarjanovic/price-volume-data-for-all-us-stocks-etfs). You will need to copy the Stocks folder in the zip file of your project home folder.

In [0]:
# Please change this with your own api key
my_key = '4TZ6V9EQSE010E4M'

Before we go ahead and start downloading the Stock prices data, you need to know that stock prices data usually come in several following categories:

- Open: Opening stock price of the day
- Close: Closing stock price of the day
- High: Highest stock price of the data
- Low: Lowest stock price of the day


**1. Getting Data from Alphavantage**

You will first load in the data from Alpha Vantage. 
1. Since you're going to make use of the American Airlines Stock market prices to make your predictions, you set the ticker to "AAL".
2. Additionally, you also define a url_string, which will return a JSON file with all the stock market data for American Airlines within the last 20 years, and a file_to_save, which will be the file to which you save the data.
3. You'll use the ticker variable that you defined beforehand to help name this file.

Next, you're going to specify a condition: if you haven't already saved data, you will go ahead and grab the data from the URL that you set in url_string; You'll store the date, low, high, volume, close, open values to a pandas DataFrame df and you'll save it to file_to_save. However, if the data is already there, you'll just load it from the CSV.

**2. Getting Data from Kaggle**

Data found on Kaggle is a collection of processed csv files and you don't have to do any preprocessing from you end, so you can directly load the data into a Pandas DataFrame and use it.



In [60]:
def download_data(data_source='kaggle', ticker= 'AAL'):
  
    if data_source=='AlphaVanatage':

          # ====================== Loading Data from Alpha Vantage ==================================
          api_key = my_key

          # JSON file with all the stock market data for AAL from the last 20 years
          url_string = "https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=%s&outputsize=full&apikey=%s"%(ticker,api_key)

          # file to Save the data
          file_to_save = 'stock_market_data-%s.csv'%ticker
          
          # If you haven't already saved data,
          if  os.path.exists(file_to_save):

              print("Downloading data from AphaVantage......")
              # Go ahead and grab the data from the url
              with urllib.request.urlopen(url_string) as url:
                    data = json.loads(url.read().decode())

                    # Extracting daily time series data of stock into a dictionary with keys as DateTime and values as resepective stock price
                    data = data['Time Series (Daily)']     

                    # Create empty dataframe to store Data
                    stock_df = pd.DataFrame(columns=['Date','Low','High','Open', 'Close'])

                    # And storing date, low, high, volume, close, open values from dictionary to a Pandas DataFrame
                    for k,v in data.items():
                          # Extracting and Converting date into suitable date time format
                          
                          key = dt.datetime.strptime(k, '%Y-%m-%d')
                          # and stock values
                          values = [key.date(),float(v['3. low']),float(v['2. high']),float(v['1. open']), float(v['4. close'])]

                          # print(key,values)

                          # Storing the data values in our empty dataframe one by one 
                          stock_df = stock_df.append({ 'Date': values[0], 'Low':values[1], 'High':values[2], 'Open':values[3], 'Close': values[4] }, ignore_index= True )

                    print('Data saved to : %s'%file_to_save) 
                    stock_df.to_csv(file_to_save) 

          
          
          # If the data is already there, just load it from the CSV
          else:
              print('File already exists. Loading data from CSV')
              stock_df = pd.read_csv(file_to_save)
    

  

    elif data_source=='Kaggle':

        # ====================== Loading Data from Kaggle ==================================  
        # You will be using HP's data. Feel free to experiment with other data.
        stock_df = pd.read_csv(os.path.join('Stocks','hpq.us.txt'),delimiter=',',usecols=['Date','Open','High','Low','Close'])
        # But while doing so, be careful to have a large enough dataset and also pay attention to the data normalization
        print('Loaded data from the Kaggle repository')

      

    
    else:
        print("Invalid Data Source")
    
    
    return stock_df



# ;et's Call the function and store the dataframe
stock_df = download_data(data_source='AlphaVanatage', ticker= 'AAL')

# printing top 5 rows of dataframe
stock_df.head()

Data saved to : stock_market_data-AAL.csv


,Date,Low,High,Open,Close
0,2020-01-28,26.26,27.1150,26.36,26.90
1,2020-01-27,25.37,26.6000,26.57,26.11
2,2020-01-24,27.01,28.8612,28.85,27.64
3,2020-01-23,26.04,28.9900,27.39,28.80
4,2020-01-22,27.30,27.9100,27.51,27.32


In [64]:
stock_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3608 entries, 0 to 3607
Data columns (total 5 columns):
Date     3608 non-null object
Low      3608 non-null float64
High     3608 non-null float64
Open     3608 non-null float64
Close    3608 non-null float64
dtypes: float64(4), object(1)
memory usage: 141.1+ KB
